<small><i>Updated February 2022 - This notebook was created by [Santi Seguí](https://ssegui.github.io/). </i></small>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\loren\AppData\Local\Temp\ipykernel_14836\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


<div class="alert alert-success" style = "border-radius:10px;border-width:3px;border-color:darkgreen;font-family:Verdana,sans-serif;font-size:16px;"><a class="anchor" id="what-is-a-recommender"></a><h3>Non-Personalized recommeder systems</h3><br></div>

<br>
<p>A non-personalized recommender system is one that makes the same recommendations for everyone. </p>

The simplest example is a retailer that shows the ten (or some number) most popular products on their homepage. <br>
Some examples: <br><br>
IMDB: MOVIE RANKING
![alt IMDB](images/np1.png)

___
Amazon: Top Recommendations
![alt Amazon](images/np2.png)
___
Amazon: Product Association
![alt Amazon](images/np3.png)
___
Reedit: News Recommendations
![alt Reedit](images/np4.png)


<p><b>Several</b> cases but <b>two main</b> approaches

1. Aggregated opinion recommenders
2. Basic product association recommenders
<br>
<div class="alert alert-success" style = "border-radius:10px;border-width:3px;border-color:darkgreen;font-family:Verdana,sans-serif;font-size:16px;"><a class="anchor" id="what-is-a-recommender"></a><h3>Aggregated opinion recommenders</h3><br></div>

Usually, the problem posed as a learning to rank problem. But what seems to be straighfowrard becomes a really complicated question: <b>How do you rank your rated items and which logic to use to display them?</b>

In order to score/rank items we first have to <b>understand the business case</b>. Of course, several factors plays a role. For instance, 

* Which information do we have about the items? Bought / Seen / Rated / ... 
* From how many users do we have the info for a particular item 
* How old is that info? 


## EXAMPLE: Non-Personalised Recommender using MovieLens Dataset
We will work with the well known MovieLens dataset (http://grouplens.org/datasets/movielens/). This dataset was initially constructed to support participants in the Netflix Prize. Today, we can find several versions of this dataset with different amout of data, from 100k samples version to 20m sample version. Although performance on bigger dataset is expected to be better, we will work with the smallest dataset: MovieLens 100K Dataset (ml-100k-zip). Working with this lite version has the benefit of less computational costs

With a unix machine the dataset can be downloaded with the following code:



In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip 
!unzip ml-1m.zip -d "data/"

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


If you are working with a windows machine, please go to the website and download the ml-1m version and extract it to the subdirectory named "data/ml-1m/"

Once you have downloaded and unzipped the file into a directory, you can create a DataFrame with the following code:

In [7]:
#NETFLIX REAL 50.000.000 usuaris and 100.000 items
%autosave 150
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt

# Load Data set
u_cols = ['user_id', 'sex', 'age', 'occupation', 'zip_code']
users = pd.read_csv('./data/ml-1m/users.dat', sep='::', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('./data/ml-1m/ratings.dat', sep='::', names=r_cols)

# the movies file contains columns indicating the movie's genres
# let's only load the first three columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date',]
movies = pd.read_csv('./data/ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3), encoding='latin-1')

# Construcció del DataFrame
data = pd.merge(pd.merge(ratings, users), movies)
data = data[['user_id','title', 'movie_id','rating','release_date','sex','age']]


print("La BD has "+ str(data.shape[0]) +" ratings")
print("La BD has ", data.user_id.nunique()," users")
print("La BD has ", data.movie_id.nunique(), " movies")
data.head()


Autosaving every 150 seconds


<ipython-input-7-f75c43d64d91>:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv('./data/ml-1m/users.dat', sep='::', names=u_cols)
<ipython-input-7-f75c43d64d91>:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('./data/ml-1m/ratings.dat', sep='::', names=r_cols)
<ipython-input-7-f75c43d64d91>:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('./data

La BD has 1000209 ratings
La BD has  6040  users
La BD has  3706  movies


,user_id,title,movie_id,rating,release_date,sex,age
0,1,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama,F,1
1,2,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama,M,56
2,12,One Flew Over the Cuckoo's Nest (1975),1193,4,Drama,M,25
3,15,One Flew Over the Cuckoo's Nest (1975),1193,4,Drama,M,25
4,17,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama,M,50


If you explore the dataset in detail, you will see that it consists of:

100,000 ratings from 943 users of 1682 movies. Ratings are from 1 to 5.
Each user has rated at least 20 movies.
Simple demographic info for the users (age, gender, occupation, zip)

### 2.1 Top movies ranking. 
The simplest way to show the ranking is by using the mean rating.

In [8]:
mean_score = data.groupby(['title'])[['rating']].mean().rename(columns = {'rating': 'mean_rating'})
mean_score.sort_values(by='mean_rating',ascending=False).head(10)

,mean_rating
title,
Ulysses (Ulisse) (1954),5.0
Lured (1947),5.0
Follow the Bitch (1998),5.0
Bittersweet Motel (2000),5.0
Song of Freedom (1936),5.0
One Little Indian (1973),5.0
Smashing Time (1967),5.0
Schlafes Bruder (Brother of Sleep) (1995),5.0
"Gate of Heavenly Peace, The (1995)",5.0


------------------------------

**What do you think about the output?**

Now, let's show only ranking the mean rating but using only those movies with at least 20 ratings

In [9]:
mean_score = data.groupby(['title'])[['rating','title']].agg({'rating':'mean',
                                                              'title':'count'}).rename(columns = {'rating': 'mean_rating','title':'num_ratings'})
mean_score[mean_score['num_ratings']>1000].sort_values(by='mean_rating',ascending=False).head(10)

,mean_rating,num_ratings
title,,
"Shawshank Redemption, The (1994)",4.554558,2227
"Godfather, The (1972)",4.524966,2223
"Usual Suspects, The (1995)",4.517106,1783
Schindler's List (1993),4.510417,2304
Raiders of the Lost Ark (1981),4.477725,2514
Rear Window (1954),4.476190,1050
Star Wars: Episode IV - A New Hope (1977),4.453694,2991
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),4.449890,1367
Casablanca (1942),4.412822,1669


------
**Any other idea?**

How can you improve it?</div>

In [26]:
k=10
mean_rating = data.rating.mean()
mean_score['damped_mean'] = (mean_score['mean_rating']*mean_score['num_ratings'] + k*mean_rating) / (k + mean_score['num_ratings']) 


In [29]:
mean_score.sort_values(by='damped_mean', ascending=False).head(10)

,mean_rating,num_ratings,damped_mean
title,,,
"Shawshank Redemption, The (1994)",4.554558,2227,4.550208
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.560510,628,4.545166
"Godfather, The (1972)",4.524966,2223,4.520741
"Usual Suspects, The (1995)",4.517106,1783,4.511888
"Close Shave, A (1995)",4.520548,657,4.506470
Schindler's List (1993),4.510417,2304,4.506403
"Wrong Trousers, The (1993)",4.507937,882,4.497551
Sanjuro (1962),4.608696,69,4.478679
Raiders of the Lost Ark (1981),4.477725,2514,4.474174


<div class="alert alert-success" style = "border-radius:10px;border-width:3px;border-color:darkgreen;font-family:Verdana,sans-serif;font-size:16px;"><a class="anchor" id="what-is-a-recommender"></a><h3>Basic product association recommenders
</h3><br> People who buy X also buy Y. </div>


In [30]:
#Let's read a dataset which contains several market baskets lists

# read data/grocieries.csv
def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

market_data = []
cont = 0
items = []
with open("./data/groceries.csv") as f:
    for l in f:
        market_data.append(l.rstrip().split(','))
        items = union(items,l.rstrip().split(','))

print("Number of different items", len(items))
print("Number of rows ", len(market_data))


print("An example:", market_data[3])

Number of different items 171
Number of rows  9835
An example: ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads']


One of the most simple ways to found association between product could be obtained as follows: $$score(Y|X) = \frac{X \ and \ Y}{X}$$

In [11]:
# Which is the top associated product with "yogurt"?

In [31]:
def top_associated_products(product,N = 5):
    d = {}
    times = 0
    for l in market_data:
        if product in l:
            times = times + 1
            for i in l:
                if i != product: 
                    if(i in d):
                        d[i] += 1.0
                    else:
                        d[i] = 1.0

    for k in d:
        d[k] =   d[k] / times
    sorted_list=sorted(d.items(), key=lambda x: x[1],reverse=True)
    return sorted_list[:N]

In [32]:
s = top_associated_products('yogurt',N = 3)
print(s)

[('whole milk', 0.40160349854227406), ('other vegetables', 0.3112244897959184), ('rolls/buns', 0.24635568513119532)]


In [33]:
# Which is the top associated prouct with "rice"?
s = top_associated_products('rice',N = 3)
print(s)

[('whole milk', 0.6133333333333333), ('other vegetables', 0.52), ('root vegetables', 0.41333333333333333)]


In [34]:
# Which is the top associated prouct with "rum"?
s = top_associated_products('rum',N = 3)
print(s)

[('whole milk', 0.38636363636363635), ('other vegetables', 0.3409090909090909), ('tropical fruit', 0.20454545454545456)]


## Exercise
What happens? Is it a good measure? It has a problem with popular items...
<br>
Let's check this other formula:
$$score(Y|X) = \frac{ \frac{X \ and \ Y}{X}} {  \frac{!X \ and \ Y}{!X} }  $$

In [39]:
from collections import defaultdict

def top_associated_products2(product,N = 5):
    d = {}
    times = 0
    for l in market_data:
        if product in l:
            times = times + 1
            for i in l:
                if i != product: 
                    if(i in d):
                        d[i] += 1.0
                    else:
                        d[i] = 1.0

    for k in d:
        d[k] =   d[k] / times
    sorted_list=sorted(d.items(), key=lambda x: x[1],reverse=True)
    return sorted_list[:N]

In [40]:
s = top_associated_products2('yogurt',N = 3)
print(s)

[('whole milk', 0.40160349854227406), ('other vegetables', 0.3112244897959184), ('rolls/buns', 0.24635568513119532)]


In [41]:
# Which is the top associated prouct with "rice"?
s = top_associated_products2('rice',N = 3)
print(s)

[('whole milk', 0.6133333333333333), ('other vegetables', 0.52), ('root vegetables', 0.41333333333333333)]


In [42]:
# Which is the top associated prouct with "rum"?
s = top_associated_products2('rum',N = 3)
print(s)

[('whole milk', 0.38636363636363635), ('other vegetables', 0.3409090909090909), ('tropical fruit', 0.20454545454545456)]


#### Let's check this last formula:
$$ score(Y|X) = \frac{P(X \ and \ Y)}{P(X)P(Y) }   $$

In [20]:
def top_associated_products3(product,N = 5):
    return 0

In [21]:
s = top_associated_products3('yogurt',N = 3)
print(s)

0


In [22]:
# Which is the top associated prouct with "rice"?
s = top_associated_products3('rice',N = 3)
print(s)

0


In [23]:
# Which is the top associated prouct with "rice"?
s = top_associated_products3('rum',N = 3)
print(s)

0


In [24]:
s = top_associated_products3('baby food',N = 3)
print(s)

0


## APRIORI Algorithm
Typically, association rules are considered interesting if they satisfy both a minimum support threshold and a minimum confidence threshold

![alt apriori](images/apriori.png)

<b>Apriori principle</b>: Any subset of a frequent itemset must be frequent

> Step 1: Find the frequent itemsset: the set of items that have minimum support.
> -  A subset of a frequent itemset must also be a frequent itemset  i.e. if {1,2} is a frequent itemset, both {1} and {2} should be a frequent itemset
> - Iteratively find frequent itemsets with cardinality from 1 to k (k-itemset)

> Step 2: Use the frequent itemsets to generate association rules

![alt apriori2](images/apriori2.png)

Reference : 
[Fast algorithms for mining association rules](http://www-cgi.cs.cmu.edu/afs/cs.cmu.edu/Web/People/ngm/15-721/summaries/12.pdf)


In [20]:
#adapted code from https://github.com/asaini/Apriori
import sys

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser


def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet


def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules
    
    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    
    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in list(largeSet.items()):
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item)/getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)),
                                           confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key = lambda x: float(x[1])):
        print("item: %s , %.3f" % (str(item), support))
    print("\n------------------------ RULES:")
    for rule, confidence in sorted(rules, key=lambda x: float(x[1])):
        pre, post = rule
        print("Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence))


def dataFromFile(fname):
        """Function which reads from the file and yields a generator"""
        file_iter = open(fname, 'r')
        for line in file_iter:
                line = line.strip().rstrip(',')                         # Remove trailing comma
                record = frozenset(line.split(','))
                yield record

In [21]:
inFile = dataFromFile('./data/groceries.csv')
minSupport = 0.04
minConfidence = 0.2
items, rules =  runApriori(inFile, minSupport, minConfidence)
printResults(items, rules)

item: ('soda', 'whole milk') , 0.040
item: ('white bread',) , 0.042
item: ('tropical fruit', 'whole milk') , 0.042
item: ('rolls/buns', 'other vegetables') , 0.043
item: ('chicken',) , 0.043
item: ('other vegetables', 'yogurt') , 0.043
item: ('other vegetables', 'root vegetables') , 0.047
item: ('frozen vegetables',) , 0.048
item: ('root vegetables', 'whole milk') , 0.049
item: ('chocolate',) , 0.050
item: ('napkins',) , 0.052
item: ('beef',) , 0.052
item: ('curd',) , 0.053
item: ('butter',) , 0.055
item: ('whole milk', 'yogurt') , 0.056
item: ('rolls/buns', 'whole milk') , 0.057
item: ('pork',) , 0.058
item: ('coffee',) , 0.058
item: ('margarine',) , 0.059
item: ('frankfurter',) , 0.059
item: ('domestic eggs',) , 0.063
item: ('brown bread',) , 0.065
item: ('whipped/sour cream',) , 0.072
item: ('fruit/vegetable juice',) , 0.072
item: ('other vegetables', 'whole milk') , 0.075
item: ('pip fruit',) , 0.076
item: ('canned beer',) , 0.078
item: ('newspapers',) , 0.080
item: ('bottled beer'

<div class  = "alert alert-success"><b>Exercice: Create and Product Association Recommender with MovieLens Dataset</b><p>
Explain the obtained results and conclusions.
</div>

